# **LP3_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Parameters:

1. ci - Direct unit cost for all operations for product Pi. For P1: c1=0.5+0.2+0.3; c2=0.6+0.1+0.2; For P3: c3=0.4+0.27+0.20, i=1,2,3.
2. pi - Unit sales price for product Pi. For P1: p1=1.5; p2=1.80; p3=1.97, i=1,2,3.
3. sTi - Smelting time for product Pi. For P1: sT1=6; sT2=10; sT3=8, i=1,2,3 in minutes.
4. mTi - Mechanisation time for product Pi. For P1: mT1=6; mT2=3; mT3=8, i=1,2,3 in minutes.
5. aTi - Assembly and packaging time for product Pi. For P1: aT1=3; aT2=2; aT3=2, i=1,2,3 in minutes.
6. sT: 8000 min (Total smelting capacity)
7. mT: 12000 min (Total mechanisation capacity)
8. aT: 10000 min (Total assembly and packaging capacity)

Decision Variables:

1. x_i - Quantity of product Pi produced. i=1,2,3
2. y_i - Quantity of product Pi smelting operation being subcontracted. i=1,2

Objective Function:

To maximize profit:
Max Z = ∑_{i=1 to 3} (pi-c_i)x_i - ∑_(i=1 to 2) 0.2*y_i

Constraints:

1. Smelting capacity constraints:
∑_{i=1 to 3} sTi*x_i + ∑_{i=1 to 2} sTi*

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
Markdown(response2.choices[0].message.content)

Here is the Pyomo code for the optimization problem:

```Python
from pyomo.environ import *

# Create a Concrete Model instance
model = ConcreteModel()

# Define the sets
I = ['P1', 'P2', 'P3'] # Products

# Define the parameters
c = {'P1': 0.5+0.2+0.3, 'P2': 0.6+0.1+0.2, 'P3' :0.4+0.27+0.20}
p = {'P1': 1.5, 'P2': 1.80, 'P3' :1.97}
sT = {'P1': 6, 'P2': 10, 'P3' :8}
mT = {'P1': 6, 'P2': 3, 'P3' :8}
aT = {'P1': 3, 'P2': 2, 'P3' :2}
sT_total = 8000
mT_total = 12000
aT_total = 10000

# Define the decision variable
model.x = Var(I, domain=NonNegativeReals) # Quantity of product Pi to produce
model.y = Var(I, domain=NonNegativeReals) # Quantity of product Pi smelting operation being subcontracted

# Define the objective
model.obj = Objective(expr = sum((p[i]-c[i])*model.x[i] for i in I) - sum(0.2*model.y[i] for i in I), sense=maximize)

# Define the constraints
model.smelting = Constraint(expr = (sum(sT[i]*model.x[i] for i in I) + sum(sT[i]*model.y[i] for i in I)) <= sT_total)
model.mechanisation = Constraint(expr = sum(mT[i]*model.x[i] for i in I) <= mT_total)
model.assembly = Constraint(expr = sum(aT[i]*model.x[i] for i in I) <= aT_total)
model.prodQuantity = ConstraintList()
for i in I:
    model.prodQuantity.add(model.x[i] >= 0)  
    model.prodQuantity.add(model.y[i] >= 0)
model.subConQuantity = ConstraintList()
for i in I:
    model.subConQuantity.add(model.y[i] <= model.x[i])

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
for i in I:
    print('product',i,'production quantity:', model.x[i].value)
    print('product',i,'subcontracted quantity:', model.y[i].value)
print('profit :', model.obj())
```

This code will solve for the optimal production and subcontracting quantities that maximize your profit considering smelting, mechanization, assembly and packaging capacity constraints, as well as the constraint about subcontracting not exceeding production.

### **Run the code Generated by GPT4**

In [9]:
from pyomo.environ import *

# Create a Concrete Model instance
model = ConcreteModel()

# Define the sets
I = ['P1', 'P2', 'P3'] # Products

# Define the parameters
c = {'P1': 0.5+0.2+0.3, 'P2': 0.6+0.1+0.2, 'P3' :0.4+0.27+0.20}
p = {'P1': 1.5, 'P2': 1.80, 'P3' :1.97}
sT = {'P1': 6, 'P2': 10, 'P3' :8}
mT = {'P1': 6, 'P2': 3, 'P3' :8}
aT = {'P1': 3, 'P2': 2, 'P3' :2}
sT_total = 8000
mT_total = 12000
aT_total = 10000

# Define the decision variable
model.x = Var(I, domain=NonNegativeReals) # Quantity of product Pi to produce
model.y = Var(I, domain=NonNegativeReals) # Quantity of product Pi smelting operation being subcontracted

# Define the objective
model.obj = Objective(expr = sum((p[i]-c[i])*model.x[i] for i in I) - sum(0.2*model.y[i] for i in I), sense=maximize)

# Define the constraints
model.smelting = Constraint(expr = (sum(sT[i]*model.x[i] for i in I) + sum(sT[i]*model.y[i] for i in I)) <= sT_total)
model.mechanisation = Constraint(expr = sum(mT[i]*model.x[i] for i in I) <= mT_total)
model.assembly = Constraint(expr = sum(aT[i]*model.x[i] for i in I) <= aT_total)
model.prodQuantity = ConstraintList()
for i in I:
    model.prodQuantity.add(model.x[i] >= 0)
    model.prodQuantity.add(model.y[i] >= 0)
model.subConQuantity = ConstraintList()
for i in I:
    model.subConQuantity.add(model.y[i] <= model.x[i])

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
for i in I:
    print('product',i,'production quantity:', model.x[i].value)
    print('product',i,'subcontracted quantity:', model.y[i].value)
print('profit :', model.obj())

product P1 production quantity: 0.0
product P1 subcontracted quantity: 0.0
product P2 production quantity: 0.0
product P2 subcontracted quantity: 0.0
product P3 production quantity: 1000.0
product P3 subcontracted quantity: 0.0
profit : 1099.9999999999998


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [18]:
from pyomo.environ import *

# Create a Concrete Model instance
model = ConcreteModel()

# Define the sets
I = ['P1', 'P2', 'P3'] # Products

# Define the parameters
c = {'P1': 0.5+0.2+0.3, 'P2': 0.6+0.1+0.2, 'P3' :0.4+0.27+0.20}
p = {'P1': 1.5, 'P2': 1.80, 'P3' :1.97}
sT = {'P1': 6, 'P2': 10, 'P3' :8}
mT = {'P1': 6, 'P2': 3, 'P3' :8}
aT = {'P1': 3, 'P2': 2, 'P3' :2}
sT_total = 8000
mT_total = 12000
aT_total = 10000

# Define the decision variable
model.x = Var(I, domain=NonNegativeReals) # Quantity of product Pi to produce
model.y = Var(I[:-1], domain=NonNegativeReals) # Quantity of product Pi smelting operation being subcontracted

# Define the objective
model.obj = Objective(expr = sum((p[i]-c[i])*model.x[i] for i in I) - sum(0.2*model.y[i] for i in I[:-1]), sense=maximize)

# Define the constraints
model.smelting = Constraint(expr = (sum(sT[i]*model.x[i] for i in I) + sum(sT[i]*model.y[i] for i in I[:-1])) <= sT_total)
model.mechanisation = Constraint(expr = sum(mT[i]*model.x[i] for i in I) <= mT_total)
model.assembly = Constraint(expr = sum(aT[i]*model.x[i] for i in I) <= aT_total)
model.prodQuantity = ConstraintList()
for i in I:
    model.prodQuantity.add(model.x[i] >= 0)
for i in I[:-1]:
    model.prodQuantity.add(model.y[i] >= 0)
model.subConQuantity = ConstraintList()

for i in I[:-1]:
    model.subConQuantity.add(model.y[i] <= model.x[i])

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
for i in I:
    print('product',i,'production quantity:', model.x[i].value)
for j in I[:-1]:
    print('product',i,'subcontracted quantity:', model.y[j].value)
print('profit :', model.obj())

product P1 production quantity: 0.0
product P2 production quantity: 0.0
product P3 production quantity: 1000.0
product P3 subcontracted quantity: 0.0
product P3 subcontracted quantity: 0.0
profit : 1099.9999999999998
